# Your mission

You started working in a financial institution. You manager Mark is insterested in investing and trading **Bitcoin** to diversify his portfolio. So he turned to you to develop a crypto trading bot.

A **crypto trading bot** is a computer program with an underlying predictive model that automatically outputs a predicted day to day price for the next 30 days based on the history of Bitcoin market price and Blockchain variables. 

Your mission is to train an accurate predictive model with the lowest mean squared error. Mark is a very technical guy, he likes to understand all **technical details** and would like you to **compare the performances of classical models and neural-net based models**.

Your **target variable** is the Bitcoin Market price.

**Data source** : https://blockchain.info/

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import helper_functions as hf

CURDIR = os.path.dirname(os.getcwd())
DATADIR = os.path.join(CURDIR,  "data")
%matplotlib inline

ModuleNotFoundError: No module named 'helper_functions'

# Data Exploration

In this section, you are to explore the data. You are to fulfill the following tasks:
- **Univariate analysis**
    - study the Bitcoin market price time series, in terms of its trend, cycle, seasonality, and stationnarity 
    - **Tips**: you can analyse ACF, PACF plots, perform Augmented Dickey-Fuller test
- **Multivariate analysis**
    - study other blockchain variables, retrieve insights
    - study the correlation between the Bitcoin market price and other blockchain variables 
    - **Tips**: Pearson correlation

In [ ]:
df_blockchain = pd.read_csv(os.path.join(DATADIR, "df_blockchain.csv"), 
                            delimiter=",")


In [ ]:
df_blockchain.head()

In [2]:
df_blockchain.columns.sort_values().shape

NameError: name 'df_blockchain' is not defined

## Plotting the data

In [ ]:
date=df_blockchain["Date"]
temps=[i for i in range(len(np.array(date)))]
prix=df_blockchain["market-price"]

plt.plot(temps,np.array(prix))

In [ ]:
plt.plot(temps[700:],np.array(prix)[700:])
plt.xlabel("Days")
plt.ylabel("Market-Price")
plt.title("Evolution of the Market after 2013 (non-negligible period)")

## Cycle


In [ ]:
plt.plot(temps[700:1200],np.array(prix)[700:1200])
plt.xlabel("Days")
plt.ylabel("Market Price")
plt.title("First Pike")

In [ ]:
plt.plot(temps[2000:2600],np.array(prix)[2000:2600])
plt.xlabel("Days")
plt.ylabel("Market Price")
plt.title("Second Pike")

In [ ]:
plt.plot(temps[3200:],np.array(prix)[3200:])
plt.xlabel("Days")
plt.ylabel("Market Price")
plt.title("Third Pike")

## Autocorrelaiton and Partial correlation

In [ ]:
plot_acf(df_blockchain["market-price"]);

In [ ]:
plot_pacf(df_blockchain["market-price"]);

In [ ]:
# Your code here for data exploration


## Train test split

In [ ]:
X_train, X_test = hf.data_split(df_blockchain)

# Modeling with ARIMA
In this section, you are to perform some classical modelings, the suggested method here is ARIMA, but you can try other models such as ARMA, ARIMAX, SARIMAX...

## Modeling
The following code allows ARIMA modeling with one combination of (p,d,q).

In [ ]:
#If you need information about the function, you can display the docstring
hf.evaluate_arima_model?

In [ ]:
parameters = (2,1,1)
errors, predictions = hf.evaluate_arima_model(X_train["market-price"], 
                                              X_test["market-price"], parameters )
errors

## Search for the best ARIMA model
We use grid search to search for the best ARIMA parameters that gives the lowest error. This follows the Box-Jenkins methology.

In [ ]:
best_cfg, best_score = hf.arima_grid_search(X_train["market-price"], 
                                            X_test["market-price"],
                                            np.arange(1,3), np.arange(3), np.arange(3))

## Visualization
To have a better view on the difference between true and predict values, we visualize them by plotting both the signals.

In [ ]:
# prepare the dataset for plotting
predict_date = df_blockchain.iloc[-30::]["Date"]
df_predict = pd.DataFrame(zip(predict_date,
                              predictions, X_test["market-price"].values),
                          columns=["date", "predict", "true"])

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

sns.lineplot(x="date", y="true", data=df_predict, 
             ax=ax, label="y_true", markers=True )
plot_ = sns.lineplot(x="date",
             y="predict",
             data=df_predict,
             color="red",ax=ax, label="y_predict")
plot_.lines[1].set_linestyle("--")
new_ticks = plot_.get_xticklabels()
freq = 4
ax.set_xticks(np.arange(0, len(new_ticks), freq))
labels = predict_date[::freq]
ax.set_xticklabels(labels)
ax.set_ylabel("Bitcoin Market Price [USD]")
ax.set_xlabel("Test date")
ax.set_title(f"Model ARIMA{parameters}")
sns.despine()
plt.legend()
plt.tight_layout()
